In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
from sklearn.metrics import classification_report

2024-03-22 19:56:38.597597: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 19:56:38.597622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 19:56:38.598153: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
seed = 42
dataset_path = '/home/deeplearning/AI/datasets/TrashBox'
epochs = 50
batch_size = 32
img_height, img_width = 300, 300

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=seed),
    tf.keras.layers.RandomRotation(factor=0.05, seed=seed),
    tf.keras.layers.RandomContrast(factor=0.2, seed=seed)
])

In [6]:
train_ds_raw = tf.keras.utils.image_dataset_from_directory(dataset_path + '/train/sub_classes', batch_size=batch_size, image_size=(img_height, img_width))
train_ds = train_ds_raw.map(lambda img, label: (data_augmentation(img), label)).shuffle(50, seed=seed).prefetch(tf.data.AUTOTUNE)

Found 14276 files belonging to 25 classes.


In [7]:
val_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size, image_size=(img_height, img_width))

Found 1774 files belonging to 25 classes.


In [8]:
class_names = train_ds_raw.class_names
num_classes = len(class_names)

In [9]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B2(
    weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.momentum = 0.98 

In [10]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.003162, momentum=0.9, weight_decay=0.000316)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/efficientnet_v2_trash_box.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)
    
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[model_checkpoint_callback])

Epoch 1/50


2024-03-22 19:58:18.418539: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1711151906.024830  350968 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


447/447 [==============================] - 74s 123ms/step - loss: 1.4085 - accuracy: 0.6444 - val_loss: 0.5407 - val_accuracy: 0.8664
Epoch 2/50
447/447 [==============================] - 60s 121ms/step - loss: 0.5043 - accuracy: 0.8589 - val_loss: 0.3848 - val_accuracy: 0.8918
Epoch 3/50
447/447 [==============================] - 60s 121ms/step - loss: 0.3627 - accuracy: 0.8919 - val_loss: 0.3444 - val_accuracy: 0.8957
Epoch 4/50
447/447 [==============================] - 60s 121ms/step - loss: 0.2847 - accuracy: 0.9184 - val_loss: 0.3211 - val_accuracy: 0.9042
Epoch 5/50
447/447 [==============================] - 60s 122ms/step - loss: 0.2398 - accuracy: 0.9284 - val_loss: 0.3007 - val_accuracy: 0.9115
Epoch 6/50
447/447 [==============================] - 60s 121ms/step - loss: 0.1963 - accuracy: 0.9413 - val_loss: 0.3201 - val_accuracy: 0.9115
Epoch 7/50
447/447 [==============================] - 59s 120ms/step - loss: 0.1680 - accuracy: 0.9503 - val_loss: 0.3002 - val_accuracy: 0.9

In [11]:
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[model_checkpoint_callback])

Epoch 1/50
447/447 [==============================] - 59s 119ms/step - loss: 0.0132 - accuracy: 0.9964 - val_loss: 0.3252 - val_accuracy: 0.9256
Epoch 2/50
447/447 [==============================] - 59s 120ms/step - loss: 0.0153 - accuracy: 0.9957 - val_loss: 0.3376 - val_accuracy: 0.9278
Epoch 3/50
447/447 [==============================] - 59s 120ms/step - loss: 0.0113 - accuracy: 0.9968 - val_loss: 0.3508 - val_accuracy: 0.9250
Epoch 4/50
447/447 [==============================] - 59s 119ms/step - loss: 0.0121 - accuracy: 0.9965 - val_loss: 0.3345 - val_accuracy: 0.9267
Epoch 5/50
447/447 [==============================] - 59s 119ms/step - loss: 0.0116 - accuracy: 0.9973 - val_loss: 0.3328 - val_accuracy: 0.9290
Epoch 6/50
447/447 [==============================] - 59s 119ms/step - loss: 0.0113 - accuracy: 0.9968 - val_loss: 0.3275 - val_accuracy: 0.9205
Epoch 7/50
447/447 [==============================] - 59s 119ms/step - loss: 0.0111 - accuracy: 0.9973 - val_loss: 0.3817 - val_ac

In [12]:
saved_model_path = 'models/efficientnet_v2_trash_box.keras' #model_checkpoint_callback.filepath
saved_model = tf.keras.models.load_model(saved_model_path)

In [13]:
saved_model.evaluate(val_ds)

56/56 [==============================] - 3s 30ms/step - loss: 0.3199 - accuracy: 0.9340


[0.3199387788772583, 0.9340473413467407]

In [30]:
test_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/test', batch_size=batch_size, image_size=(img_height, img_width))

Found 1802 files belonging to 25 classes.


In [31]:
saved_model.evaluate(test_ds)

57/57 [==============================] - 2s 27ms/step - loss: 0.3736 - accuracy: 0.9156


[0.3736158311367035, 0.9156492948532104]

In [32]:
labels = []
predictions = []

for X, y in test_ds:
    y_pred = tf.argmax(saved_model.predict(X, verbose=0), axis=1).numpy().tolist()
    labels.extend(y.numpy().tolist())
    predictions.extend(y_pred)

In [33]:
print(classification_report(labels, predictions, target_names=test_ds.class_names))

                     precision    recall  f1-score   support

      beverage cans       0.89      0.91      0.90       100
          cardboard       0.93      0.93      0.93       242
     cigarette butt       1.00      1.00      1.00        52
 construction scrap       0.83      0.96      0.89        55
  electrical cables       0.95      0.84      0.89        70
   electronic chips       0.90      0.84      0.87        62
              glass       0.95      0.93      0.94       254
             gloves       0.92      0.98      0.95        45
            laptops       0.81      0.86      0.83        50
              masks       0.94      0.90      0.92        50
          medicines       0.94      0.96      0.95        52
   metal containers       0.85      0.88      0.87        51
         news paper       0.77      0.85      0.81        20
other metal objects       0.25      0.20      0.22         5
other paper objects       0.90      0.87      0.88       106
         paper cups    